# Day 5
Link to challenge: https://adventofcode.com/2022/day/5

## Part 1

Just as usual, we start with parsing the input. This time I decided to start writing unit-tests, as the challenges are slowly getting harder and unit-tests will significantly reduce the amount of mistakes I will make. I had a quick look on how I could implement unit-tests in Jupyter and found an [answer](https://stackoverflow.com/questions/40172281/unit-tests-for-functions-in-a-jupyter-notebook) on stackoverflow, let's try it out. I wrote all unit-tests at the end of this notebook, even though I wrote them first ([TDD](https://en.wikipedia.org/wiki/Test-driven_development)).

In [68]:
def parse_input(lines):
    lines = map(str, lines)
    stacks_fully_parsed = False
    stack_lines = []
    instruction_lines = []
    for line in lines:
        if line.strip() == "":
            stacks_fully_parsed = True
            continue
        if stacks_fully_parsed:
            instruction_lines.append(parse_instruction_line(line))
        else:
            stack_lines.append(parse_stack_line(line))
    return instruction_lines, build_stacks_from_stack_lines(stack_lines)
    

def parse_instruction_line(line: str):
    splitted = line.strip().split(" ")
    return [int(splitted[1]), int(splitted[3]), int(splitted[5])]

def parse_stack_line(line: str):
    if "[" not in line:
        return None # this is the line with the numbers. We don't care about this
    else:
        line = line.rstrip()
        stacks = []
        for idx, c in enumerate(line):
            if idx % 4 == 1:
                stacks.append(c if c != " " else None)
        return stacks

def build_stacks_from_stack_lines(stack_lines: list):
    stack_lines = list(filter(lambda l: l != None, stack_lines))
    amount_of_stacks = max(map(len, stack_lines))
    stacks = [[] for i in range(amount_of_stacks)]
    for i in range(len(stack_lines)):
        stacks = push_stack_line_to_stacks(stacks, stack_lines[-i - 1])
    return stacks

def push_stack_line_to_stacks(stacks, stack_line):
    stacks = list(stacks) # prevent side-effects
    for idx, el in enumerate(stack_line):
        if el != None:
            stacks[idx].append(el)
    return stacks

Parsing is done! 🥳 Let's start executing the instructions onto the stacks!

In [69]:
def execute_instruction_on_stacks(instruction: list, stacks: list):
    stacks = list(stacks) # to prevent side-effects
    for i in range(instruction[0]):
        stacks[instruction[2]-1].append(stacks[instruction[1]-1].pop())
    return stacks

Now we can put everything together.

In [70]:
with open("puzzle-input/5.txt", "r") as f:
    instructions, stacks = parse_input(f.readlines())
    for inst in instructions:
        stacks = execute_instruction_on_stacks(inst, stacks)

    # Print top element of each stack
    print(''.join(map(lambda stack: stack[-1], stacks)))

CNSZFDVLJ


## Part 2

Now we have to move multiple crates at once, so we modify our `execute_instruction_on_stacks`-method with a simple temporary stack. I know, this is not the most clever solution but it works and is easy to understand. For elegant solution take a look in the leaderboard ;)

In [71]:
def execute_cratemover_9001_instruction_on_stacks(instruction: list, stacks: list):
    stacks = list(stacks) # to prevent side-effects
    temp_stack = []
    for i in range(instruction[0]):
        temp_stack.append(stacks[instruction[1]-1].pop())
    for i in range(len(temp_stack)):
        stacks[instruction[2]-1].append(temp_stack.pop())
    return stacks

with open("puzzle-input/5.txt", "r") as f:
    instructions, stacks = parse_input(f.readlines())
    for inst in instructions:
        stacks = execute_cratemover_9001_instruction_on_stacks(inst, stacks)

    # Print top element of each stack
    print(''.join(map(lambda stack: stack[-1], stacks)))

QNDWLMGNS


In [72]:
import unittest

class TestNotebook(unittest.TestCase):
    
    def test_parse_instruction_line(self):
        self.assertListEqual(parse_instruction_line("move 3 from 9 to 6"), [3, 9, 6])

    def test_parse_instruction_line_multiple_digits(self):
        self.assertListEqual(parse_instruction_line("move 30 from 90 to 60"), [30, 90, 60])

    def test_parse_stack_line_1(self):
        self.assertListEqual(parse_stack_line("                        [Z] [W] [Z]"), [None, None, None, None, None, None, "Z", "W", "Z"])

    def test_parse_stack_line_2(self):
        self.assertEqual(parse_stack_line(" 1   2   3   4   5   6   7   8   9 "), None)
    
    def test_push_stack_line_to_stacks(self):
        self.assertListEqual(push_stack_line_to_stacks([["A", "B", "C"], []], ["D", None]), [["A", "B", "C", "D"], []])
    
    def test_execute_instruction_on_stacks(self):
        self.assertListEqual(execute_instruction_on_stacks([2, 1, 3], [["A", "B"], [], ["C"]]), [[], [], ["C", "B", "A"]])
    
    def test_execute_cratemover_9001_instruction_on_stacks(self):
        self.assertListEqual(execute_cratemover_9001_instruction_on_stacks([2, 1, 3], [["A", "B"], [], ["C"]]), [[], [], ["C", "A", "B"]])

unittest.main(argv=[''], verbosity=2, exit=False)

test_execute_cratemover_9001_instruction_on_stacks (__main__.TestNotebook) ... ok
test_execute_instruction_on_stacks (__main__.TestNotebook) ... ok
test_parse_instruction_line (__main__.TestNotebook) ... ok
test_parse_instruction_line_multiple_digits (__main__.TestNotebook) ... ok
test_parse_stack_line_1 (__main__.TestNotebook) ... ok
test_parse_stack_line_2 (__main__.TestNotebook) ... ok
test_push_stack_line_to_stacks (__main__.TestNotebook) ... ok

----------------------------------------------------------------------
Ran 7 tests in 0.005s

OK
